Lab 3

Emily Goodman

In [83]:
import numpy as np
import pandas as pd

In [84]:
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")
xmas.head()

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN


Function 1

In [85]:
def pluralize_gift(gift):
  """
  Returns plural of a noun

  Parameters
  ----------
  gift: str
    A noun

  Return
  ------
  str
    Plural version
  """

  if "oo" in gift:
    gift = gift.replace("oo", "ee")
  elif gift[-1]=="y":
    gift = gift.replace("y", "ies")
  elif gift[-1]=="s":
    gift = gift.replace("s", "s")
  else:
    gift = gift + "s"

  return gift

In [86]:
## test
# Should work
pluralize_gift("goose")

# Will work if your function is vectorized!
#pluralize_gift(xmas['Gift.Item'])
(xmas['Gift.Item'].apply(pluralize_gift))


,Gift.Item
0,partridges
1,doves
2,hens
3,birds
4,rings
5,geese
6,swans
7,maids
8,ladies
9,lords


Function 2

In [87]:
import inflect
p= inflect.engine()
def make_phrase(num, item, verb, adjective, location):

  """
   Returns phrase with given parameters

  Parameters
  ----------
  num: int
    Number of items
  num word: str
    Word for number
  item: str
    gift item
  verb: str or pd.series
    Verb with said item/gift
  adjective: str or pd.series
    Adjective assoacited with given item/gift
  location: str or pd.series
    Location for given item/gift

  Return
  ------
  str
    phrase for the item/gift
  """

  ## Step 1: Replace NAs with blank strings
  if pd.isnull(verb):
    verb = ""
  if pd.isnull(adjective):
    adjective = ""
  if pd.isnull(location):
    location = ""

  ## Step 2: If the day number is larger than 1, the gift items need pluralized!
  ### Hint: call the function you created above!
  if num>1:
    item = pluralize_gift(item)
    num_word= p.number_to_words(num)

  ## Step 3: Figure out if a gift item starts with a vowel
  vowel=["A","E","I","O","U","a","e","i","o","u"]
  ## Step 4: For the first day, if the gift item starts with a vowel, replace the day with "an" and if the gift item does not start with a vowel, replace the day with "a" (e.g. a partridge in a pear tree). If it is not the first day, use just the number word (e.g. ten lords a leap)
  if num == 1:
    if item[0] in vowel:
      num_word = "an"
    else:
      num_word= "a"
   ## Step 5: Put all of the pieces together into one string and return!
  phrase = num_word +" " + adjective +" "+ item  + " " + verb + location

  return phrase.replace("  ", " ").strip()



In [88]:
#check to see if it works
make_phrase(num=10,
            item = "lords",
            verb = "a-leaping",
            adjective = "",
            location = "")

'ten lords a-leaping'

In [90]:
#test 2 to see if it works
make_phrase(num=2,
            item = "Turtle Doves",
            verb = "",
            adjective = "",
            location = "")

'two Turtle Doves'

In [91]:
#make full phrase column
xmas['full_phrase']= xmas.apply(lambda x: make_phrase(x['Day'],
                                          x['Gift.Item'],
                                          x['Verb'],
                                          x['Adjective'],
                                          x['Location']), axis=1)

In [92]:
xmas.head()

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,full_phrase
0,1,first,partridge,NaN,NaN,in a pear tree,a partridge in a pear tree
1,2,second,dove,NaN,turtle,NaN,two turtle doves
2,3,third,hen,NaN,french,NaN,three french hens
3,4,fourth,bird,NaN,calling,NaN,four calling birds
4,5,fifth,ring,NaN,golden,NaN,five golden rings


In [93]:
#check the make phrase
make_phrase(xmas.iloc[1]["Day"],
            xmas.iloc[1]["Gift.Item"],
            xmas.iloc[1]["Verb"],
            xmas.iloc[1]["Adjective"],
            xmas.iloc[1]["Location"],
)

'two turtle doves'

Function 3

In [94]:
import inflect
def sing_day(dataset, num, phrase_col):
  """
  Return the song for the given day
  Parameters
  ----------
  dataset: dataframe
    The dataframe with the song lyrics
  num: int
    Number of items
  phrase_col: str
    Name of the column with the phrase
  Return
  ------
  str
    Song for the given day
  """
  dataset['full_phrase']= dataset.apply(lambda x: make_phrase(x['Day'],
                                          x['Gift.Item'],
                                          x['Verb'],
                                          x['Adjective'],
                                          x['Location']), axis=1)

  p= inflect.engine()
  # Step 1: Setup the intro line
  verses = []

    # Step 1: Loop through each day from 1 to the specified 'num' day
  for current_day in range(1, num + 1):
      num_day = p.ordinal(p.number_to_words(current_day))
      intro = "On the " + num_day + " day of Christmas, my true love sent to me:"

        # Step 2: Sing the gift phrases for the current day and all previous days (in reverse order)
      gifts = ""
      for i in range(current_day, 0, -1):  # For each day, going backward
          gift_line = dataset.iloc[i - 1][phrase_col]

            # Handle "and" for the last item on days 2 and beyond
          if i == 1 and current_day > 1:
              gifts += "and " + gift_line + "."
          else:
              gifts += gift_line + ",\n" if i > 1 else gift_line + "."

        # Combine intro and gifts for the current day
      verse = intro + "\n" + gifts.strip()
      verses.append(verse)

    # Step 3: Return all verses joined with two newlines
  return "\n\n".join(verses)

In [97]:
#test to see if it works. have to use print to use \n
print(sing_day(xmas, 3, "full_phrase"))

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree.

On the second day of Christmas, my true love sent to me:
two turtle doves,
and a partridge in a pear tree.

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.


In [98]:
#print all days
print(sing_day(xmas, 12, "full_phrase"))

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree.

On the second day of Christmas, my true love sent to me:
two turtle doves,
and a partridge in a pear tree.

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fourth day of Christmas, my true love sent to me:
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fifth day of Christmas, my true love sent to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the sixth day of Christmas, my true love sent to me:
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the seventh day of Christmas, my true love sent to me:
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a 

Function 4

In [99]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")
xmas2.head()

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,email,NaN,NaN,from Cal Poly
1,2,second,point,NaN,meal,NaN
2,3,third,pen,NaN,lost,NaN
3,4,fourth,review,NaN,course,NaN
4,5,fifth,exam,NaN,practice,NaN


In [100]:
#make full phrase column for xmas 2
xmas2['full_phrase']= xmas2.apply(lambda x: make_phrase(x['Day'],
                                          x['Gift.Item'],
                                          x['Verb'],
                                          x['Adjective'],
                                          x['Location']), axis=1)

In [101]:
#run the sing day on xmas two
print(sing_day(xmas2, 12, "full_phrase"))

On the first day of Christmas, my true love sent to me:
an email from Cal Poly.

On the second day of Christmas, my true love sent to me:
two meal points,
and an email from Cal Poly.

On the third day of Christmas, my true love sent to me:
three lost pens,
two meal points,
and an email from Cal Poly.

On the fourth day of Christmas, my true love sent to me:
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the fifth day of Christmas, my true love sent to me:
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the sixth day of Christmas, my true love sent to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the seventh day of Christmas, my true love sent to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On t